In [1]:
import yaml
import glob
import re




In [5]:
dir_path = r'..\schema-205\schema-source\rs*.schema.yaml'
files = glob.glob(dir_path)
print(files)


['..\\schema-205\\schema-source\\RS0001.schema.yaml', '..\\schema-205\\schema-source\\RS0002.schema.yaml', '..\\schema-205\\schema-source\\RS0003.schema.yaml', '..\\schema-205\\schema-source\\RS0004.schema.yaml', '..\\schema-205\\schema-source\\RS0005.schema.yaml', '..\\schema-205\\schema-source\\RS0006.schema.yaml', '..\\schema-205\\schema-source\\RS0007.schema.yaml']


In [3]:
rs_list = []
schemas = {}
for file in files:
    with open(file, 'r') as input_file:
        rs = file[-18:][0:6]
        rs_list.append(rs)
        print(rs)
        schemas[rs] = yaml.load(input_file, Loader=yaml.FullLoader)


RAE205
RS0001
RS0002
RS0003
RS0004
RS0005
RS0006
RS0007


In [4]:
rs_list

['RAE205',
 'RS0001',
 'RS0002',
 'RS0003',
 'RS0004',
 'RS0005',
 'RS0006',
 'RS0007']

In [4]:
def extract_itemnames(string):
    substring = string.strip("()").replace("{","").replace("}","").replace(" ","")
    item = substring.split(",")
    return item

extract_itemnames("({stuff}, {stuff2})")

['stuff', 'stuff2']

In [42]:

def extract_grid_variables(rs, map):

    grid_var_list = rs[map]["Data Elements"]["grid_variables"]["Data Type"].strip("{}")
    grid_variables={}
    for var in list(rs[grid_var_list]["Data Elements"].keys()):
        grid_variables[var] = {"units": rs[grid_var_list]["Data Elements"][var]["Units"],
                            "short_name": shorten_name(var)}

    return grid_variables

extract_grid_variables(schemas["RS0004"],"PerformanceMapCooling")

{'outdoor_coil_entering_dry_bulb_temperature': {'units': 'K',
  'short_name': 'out_coi_ent_dry_bul_tem'},
 'indoor_coil_entering_relative_humidity': {'units': '-',
  'short_name': 'ind_coi_ent_rel_hum'},
 'indoor_coil_entering_dry_bulb_temperature': {'units': 'K',
  'short_name': 'ind_coi_ent_dry_bul_tem'},
 'indoor_coil_air_mass_flow_rate': {'units': 'kg/s',
  'short_name': 'ind_coi_air_mas_flo_rat'},
 'compressor_sequence_number': {'units': '-', 'short_name': 'com_seq_num'},
 'ambient_absolute_air_pressure': {'units': 'Pa',
  'short_name': 'amb_abs_air_pre'}}

In [6]:
def camel_to_snake_case(camel):
    # From https://www.geeksforgeeks.org/python-program-to-convert-camel-case-string-to-snake-case/
     
    snake = ''.join(['_'+i.lower() if i.isupper()
               else i for i in camel]).lstrip('_')

    return snake

In [8]:
rs_dict = {}
for rs in rs_list:
    rs_dict[rs]={}
    perf_elements = list(schemas[rs]["Performance"]["Data Elements"].keys())
    for element in perf_elements:
        if "performance_map" in element:
            maps = extract_itemnames(schemas[rs]["Performance"]["Data Elements"][element]["Data Type"])
            for map in maps:
                snake_map = camel_to_snake_case(map)
                rs_dict[rs][snake_map]=extract_grid_variables(schemas[rs],map)

rs_dict

{'RS0001': {'performance_map_cooling': {'evaporator_liquid_volumetric_flow_rate': 'm3/s',
   'evaporator_liquid_leaving_temperature': 'K',
   'condenser_liquid_volumetric_flow_rate': 'm3/s',
   'condenser_liquid_entering_temperature': 'K',
   'compressor_sequence_number': '-'},
  'performance_map_standby': {'environment_dry_bulb_temperature': 'K'}},
 'RS0002': {},
 'RS0003': {'performance_map_continuous': {'standard_air_volumetric_flow_rate': 'm3/s',
   'static_pressure_difference': 'Pa'},
  'performance_map_discrete': {'speed_number': '-',
   'static_pressure_difference': 'Pa'}},
 'RS0004': {'performance_map_cooling': {'outdoor_coil_entering_dry_bulb_temperature': 'K',
   'indoor_coil_entering_relative_humidity': '-',
   'indoor_coil_entering_dry_bulb_temperature': 'K',
   'indoor_coil_air_mass_flow_rate': 'kg/s',
   'compressor_sequence_number': '-',
   'ambient_absolute_air_pressure': 'Pa'},
  'performance_map_standby': {'outdoor_coil_environment_dry_bulb_temperature': 'K'}},
 'RS00

In [9]:
rs_dict

{'RS0001': {'performance_map_cooling': {'evaporator_liquid_volumetric_flow_rate': 'm3/s',
   'evaporator_liquid_leaving_temperature': 'K',
   'condenser_liquid_volumetric_flow_rate': 'm3/s',
   'condenser_liquid_entering_temperature': 'K',
   'compressor_sequence_number': '-'},
  'performance_map_standby': {'environment_dry_bulb_temperature': 'K'}},
 'RS0002': {},
 'RS0003': {'performance_map_continuous': {'standard_air_volumetric_flow_rate': 'm3/s',
   'static_pressure_difference': 'Pa'},
  'performance_map_discrete': {'speed_number': '-',
   'static_pressure_difference': 'Pa'}},
 'RS0004': {'performance_map_cooling': {'outdoor_coil_entering_dry_bulb_temperature': 'K',
   'indoor_coil_entering_relative_humidity': '-',
   'indoor_coil_entering_dry_bulb_temperature': 'K',
   'indoor_coil_air_mass_flow_rate': 'kg/s',
   'compressor_sequence_number': '-',
   'ambient_absolute_air_pressure': 'Pa'},
  'performance_map_standby': {'outdoor_coil_environment_dry_bulb_temperature': 'K'}},
 'RS00

In [43]:
def shorten_name(string):
    # routine to shorten a snake case name to first 3 letters of each word
    short_words = [word[:3] for word in string.split("_")]
    short_string = "_".join(short_words)

    return short_string

shorten_name("outdoor_coil_entering_dry_bulb_temperature")

'out_coi_ent_dry_bul_tem'

In [56]:
def shorten_name2(string):
    # routine to shorten a snake case name to first 3 letters of each word

    # do this with pattern matching for a "smart" shortening
    dict={"dry_bulb_temperature": "Tdb",
         "wet_bulb_temperature": "Twb",
         "indoor_coil": "in_coil",
         "outdoor_coil": "out_coil",
         "absolute_air_pressure": "Pabs",
         "mass_flow_rate": "Mdot",
         "volumetric_flow_rate": "Vdot",
         "pressure_difference": "Pdiff",
         "entering": "ent",
         "leaving": "leav",
         "exiting": "exit",
         "ambient": "amb",
         "power": "pow",
         "frequency": "freq",
         "relative_humidity": "RH",
         "indoor": "in",
         "outdoor": "out",
         "total": "tot",
         "latent": "lat",
         "sensible": "sens",
         "capacity": "cap",
         "enviornment": "env",
         "compressor": "comp",
         "sequence": "seq",
         "number": "num",
         "rotational": "rot",
         "auxiliary": "aux",
         "evaporator": "evap",
         "condenser": "cond",
         "liquid": "liq",
         "standard": "std",
         "static": "stat",
         "pressure": "pres",
         "difference": "diff",
         "impeller": "imp",
         "input": "in",
         "output": "out",
         }
    short_string = string
    for word in dict:
        short_string = re.sub(word, dict[word], short_string)

    return short_string

shorten_name2("outdoor_coil_entering_dry_bulb_temperature")

'out_coil_ent_Tdb'

In [24]:
a = "test_word_happy_and_veryhappy"
a.rsplit("_")

['test', 'word', 'happy', 'and', 'veryhappy']

In [57]:
rs_in

NameError: name 'rs_in' is not defined